## Importation des librairies

In [46]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import seaborn as sns 
from pymongo import MongoClient
import openpyxl
from openpyxl import load_workbook

## Initialisation du Token

The __get_token_from_file__ fonction
   
Description : Retrieve the token from the token.json file. This token has renewed instantly in the  /admindata.atmo-france.org/api/doc website by clik to a button.
    
    Args:
    - file_path (str): Path to the token.json file.
    
    Returns:
    - str: The token value.
    """

In [47]:
def get_token_from_file(file_path):

    with open(file_path, 'r') as file:
        data = json.load(file)
        return data.get('token', None)
    
api_key = get_token_from_file('token.json')

## Fonctions Secondaires

   The __fetch_atmo_data__ fonction
    
    
   Description : Fetches data from AtmoData API.
    
    Args:
    - api_key (str): API key for authentication.
    - id_data (int): Identifier of the data.
    - params (dict): Parameters for the query.
    
    Returns:
    - tuple: Tuple containing the DataFrame of JSON response from the API and the response status code.
    
    

In [48]:
def fetch_atmo_data(api_key, id_data, params):

    # Construction de l'URL avec l'identifiant de la donnée
    url = f"https://admindata.atmo-france.org/api/data/{id_data}/"
    
    # Convertir les paramètres en une chaîne JSON et les ajouter à l'URL
    url += json.dumps(params)
    
    url += "?withGeom=false"
    
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    
    response = requests.get(url, headers=headers)
       
    if response.status_code == 200:
        data = json.loads(response.content)
        return pd.json_normalize(data['features']), response.status_code
    elif response.status_code == 401 or response.status_code == 400: 
        #sys.exit(f"Erreur: {response.status_code}") 
        return None, response.status_code
    else:
        return None, response.status_code
        #sys.exit(f"Erreur: {response.status_code}")  


   The __add_data_to_database()__ function
    """
    Add data to MongoDB collection based on id_data.
    
    Args:
    - collection (pymongo.collection.Collection): MongoDB collection.
    - data_dict (dict): Dictionary containing data to be added to the collection.
    - id_data (int): Identifier of the data.

In [49]:
def add_data_to_database(data_dict, id_data):

    if id_data == 112:
        for entry in data_dict:
            aire_quality.insert_one(entry)
    elif id_data == 113 :
        for entry in data_dict:
            pollution_113.insert_one(entry)
            
    elif  id_data == 114:
        for entry in data_dict:
            pollution_114.insert_one(entry)
            
    elif  id_data == 119:
        for entry in data_dict:
            emission_region.insert_one(entry)



The __write_trace_data_file__() function
    
   Description : Writes information about zone (commune/region) without data to a trace file.
    
    Args:
    - code_commune (int): Code of the commune.
    - id_data (int): Identifier of the data.
    - start_date (str): Start date in the format "YYYY-MM-DD".
    - stop_date (str): Stop date in the format "YYYY-MM-DD".
    - trace_file (str): Path to the CSV file to store the trace of communes without results.
    

In [50]:
def write_trace_data_to_file(code_zone, id_data, start_date, trace_file):

    # Create DataFrame with trace data
    trace_data = {
        "code_zone": code_zone,
        "id_data": id_data,
        "start_date": start_date
    }
    trace_df = pd.DataFrame([trace_data])
    
    # Write trace data to CSV file
    trace_df.to_csv(trace_file, mode="a", index=False, header=False)


### A compléter pour les autres types de données

 The __clean_data()__ function  
   
   Description : Clean commune data DataFrame by removing "geometry" column and renaming columns starting with "properties.".
    
    Args:
    - commune_data (pd.DataFrame): DataFrame containing commune data.
    - id_data (int): Identifier of the data.
    
    Returns:
    - pd.DataFrame: Cleaned DataFrame.
    """

In [51]:
def clean_data(commune_data, id_data):
    
    # Rename columns starting with "properties."
    commune_data.rename(columns=lambda x: x.replace("properties.", ""), inplace=True)
    if id_data == 112:
        commune_data.drop(columns=["geometry"], inplace=True, errors="ignore")
        commune_data.drop(columns=["type"], inplace=True, errors="ignore")
        commune_data.drop(columns=["x_wgs84"], inplace=True, errors="ignore")
        commune_data.drop(columns=["x_reg"], inplace=True, errors="ignore")
        commune_data.drop(columns=["y_reg"], inplace=True, errors="ignore")
        commune_data.drop(columns=["y_wgs84"], inplace=True, errors="ignore")
        commune_data.drop(columns=["type_zone"], inplace=True, errors="ignore")
        commune_data.drop(columns=["y_reg"], inplace=True, errors="ignore")
        commune_data.drop(columns=["epsg_reg"], inplace=True, errors="ignore")
        
    elif ((id_data == 113) or (id_data == 114)):
        # Remove the "type" column
        commune_data.drop(columns=["geometry"], inplace=True, errors="ignore")
        commune_data.drop(columns=["type"], inplace=True, errors="ignore")
    else :
        commune_data.drop(columns=["geometry"], inplace=True, errors="ignore")
        # A compléter attend de voir la structures des résultats avec une id_data == 119
        
    return commune_data


The __update_status()__ function

    Description :    Updates the status of the processed commune in the Excel file by marking it as processed for the specified id_data.
    
    Args:
    - code_insee_file (str): Path to the Excel file containing commune codes INSEE and department codes.
    - zone_code (int): Code of the zone to update.
    - colonne_verification (str): Column verification value ("verif_112", "verif_113", "verif_114", "verif_119").
    """

In [52]:
def update_status(code_insee_file, zone_code, colonne_verification):

    # Mapping des valeurs de colonne_verification aux indices de colonne correspondants
    colonne_indices = {
        "verif_112": 6,
        "verif_113": 7,
        "verif_114": 8,
        "verif_119": 9
    }
    
    # Vérifier si colonne_verification est une valeur valide
    if colonne_verification not in colonne_indices:
        print("Erreur: colonne_verification invalide.")
        return
    
    # Load the Excel file as a template
    wb = load_workbook(code_insee_file)
    
    # Select the active worksheet
    ws = wb.active
    
    # Find the row index corresponding to the commune code
    for idx, row in enumerate(ws.iter_rows(min_row=2, max_col=ws.max_column, max_row=ws.max_row), start=2):
        if row[1].value == zone_code:  
            # Update the status of the commune for the specified id_data (assuming id_data is the column name)
            ws.cell(row=idx, column=colonne_indices[colonne_verification]).value = 1
            break
    
    # Save the modified Excel file
    wb.save(code_insee_file)
    wb.close()


## Fonctions principales

The __store_commune()__ function

Description : Store data for a specific commune from the AtmoData API and updates its status.
    
    Args:
    - api_key (str): API key for authentication.
    - id_data (int): Identifier of the data (112 for air quality indices).
    - start_date (str): Start date in the format "YYYY-MM-DD".
    - commune_code (int): Code of the commune to fetch data for.
    - code_insee_file (str): Path to the Excel file containing commune codes INSEE and department codes.
    - trace_file (str): Path to the CSV file to store the trace of communes without results.
    """

In [53]:
def store_commune(api_key, id_data, start_date, commune_code, code_insee_file, trace_file):
    colonne_verification = "verif_" + str(id_data)
    
    # Define the parameters for the query
    params = {
        "code_zone": {"operator": "=", "value": commune_code},
        "date_ech": {"operator": ">=", "value": start_date}
    }
    
    # Fetch data for the current commune
    commune_data, status_api = fetch_atmo_data(api_key, id_data, params)
    
    # Append the data to the DataFrame
    if commune_data is not None and not commune_data.empty:
        commune_data = clean_data(commune_data, id_data)
        
        # Add to database
        add_data_to_database(commune_data.to_dict(orient='records'), id_data)
    else:
        if status_api == 400 or status_api == 401: 
            sys.exit("Renouveler le token")
        else: 
            # Write trace data to file for commune without data
            write_trace_data_to_file(commune_code, id_data, start_date, trace_file)

    # Update the status of the processed commune
    update_status(code_insee_file, commune_code, colonne_verification)


 The __store departement()__ function
 
 Description : store data for all communes in a specific department from the AtmoData API.
    
    Args:
    - api_key (str): API key for authentication.
    - id_data (int): Identifier of the data (112 for air quality indices).
    - start_date (str): Start date in the format "YYYY-MM-DD".
    - code_insee_file (str): Path to the Excel file containing commune codes INSEE and department codes.
    - code_dept (int): Code of the department.
    - trace_file (str): Path to the CSV file to store the trace of communes without results.
    """

In [54]:
def store_department(api_key, id_data, start_date, code_insee_file, code_dept, trace_file):

    colonne_verification = "verif_"+str(id_data)
    # Read commune codes INSEE and department codes from the Excel file
    df_communes = pd.read_excel(code_insee_file)
    # Filter communes for the specified department and those not processed for id_data
    dept_communes = df_communes[(df_communes[colonne_verification] == 0) & (df_communes["DEP"] == code_dept)]
    # Iterate over all communes in the specified department
    for _, commune in dept_communes.iterrows():
        commune_code = commune["COM"]
        
        # store data and update status for the current commune
        store_commune(api_key, id_data, start_date, commune_code, code_insee_file, trace_file)
    

The __store_region__ function  
"""
    Store data for all communes in all departments of a specific region from the AtmoData API.
    
    Args:
    - api_key (str): API key for authentication.
    - id_data (int): Identifier of the data (112 for air quality indices).
    - start_date (str): Start date in the format "YYYY-MM-DD".
    - region_code (int): Code of the region.
    - code_insee_file (str): Path to the Excel file containing commune codes INSEE and department codes.
    - trace_file (str): Path to the CSV file to store the trace of communes without results.
    """

In [55]:
def store_region(api_key, id_data, start_date, region_code, code_insee_file, trace_file):

    # Get the list of departments in the region from the Excel file
    df = pd.read_excel(code_insee_file)
    region_departments = df[df["REG"] == region_code]["DEP"].unique()
    
    # Iterate over all departments in the region
    for dept_code in region_departments:
        #
        api_key = get_token_from_file('token.json')
        #
        # Store data for the current department
        store_department(api_key, id_data, start_date, code_insee_file, dept_code, trace_file)


The __fetch_and_store_region()__ function :
   
Description : Fetches and store data for a specific region from the AtmoData API and updates its status.
    
    Args:
    - api_key (str): API key for authentication.
    - id_data (int): Identifier of the data (112 for air quality indices).
    - start_date (str): Start date .
    - region_code (int): Code of the region to fetch data for.
    - code_insee_file (str): Path to the Excel file containing commune codes INSEE and department codes.
    - trace_file (str): Path to the CSV file to store the trace of communes without results.
    """

In [56]:
def fetch_and_store_region(api_key, id_data, start_date, region_code, code_insee_file, trace_file):

    colonne_verification = "verif_" + str(id_data)
    
    # Define the parameters for the query
    params = {
        "code_zone": {"operator": "=", "value": int(region_code)},
        "date_ech": {"operator": ">=", "value": start_date}
    }
    
    # Fetch data for the current region
    region_data, status_api = fetch_atmo_data(api_key, id_data, params)
    
    # Append the data to the DataFrame
    if region_data is not None and not region_data.empty:
        region_data = clean_data(region_data, id_data)
        
        # Add to database
        add_data_to_database(region_data.to_dict(orient='records'), id_data)
    else:
        if status_api == 400 or status_api == 401: 
            sys.exit("Renouveler le token")
        else: 
            # Write trace data to file for region without data
            write_trace_data_to_file(region_code, id_data, start_date, trace_file)

    # Update the status of the processed region
    update_status(code_insee_file, region_code, colonne_verification)


The __general_store()__ function :

Description : Store data for all communes in all regions or for all region if id_data==119, based on the provided data identifier.
    
    Args:
    - api_key (str): API key for authentication.
    - id_data (int): Identifier of the data (112, 113, 114, or 119 for air quality indices).
    - start_date (str): Start date in the format "YYYY-MM-DD".
    - code_insee_file (str): Path to the Excel file containing commune codes INSEE and department codes.
    - trace_file (str): Path to the CSV file to store the trace of communes without results.
    """   

In [57]:
def general_store(api_key, id_data, start_date, code_insee_file, trace_file):

    colonne_verification = "verif_" + str(id_data)
    # Read commune codes INSEE, department codes, and region codes from the Excel file
    df_communes = pd.read_excel(code_insee_file)
    
    # Get unique region codes from the DataFrame
    region_codes = df_communes["REG"].unique()
    # Iterate over all unique region codes
    for region_code in region_codes:
        # Fetch data for the region based on the data identifier
        if id_data in [112, 113, 114]:
            store_region(api_key, id_data, start_date, region_code, code_insee_file, trace_file)
        elif id_data == 119:
            fetch_and_store_region(api_key, id_data, start_date, region_code, code_insee_file, trace_file)
        else:
            # Error handling for invalid id_data
            print(f"Error: Unsupported id_data {id_data}.")

## Acquision des données selon divers paramètres

Maintenant que nous avons défini nos différentes fonctions, nous allons ensuite définir les paramètres et appeler la fonction principale general_store pour stocker progressivement nos différents type de données (indice aire, pollution de l'année dernière, pollution constaté la veille, le jour même et prévu le lendemin ainsi que les émissions des régions), en ajustant juste le paramètre id_data( resp. 112, 113, 114 ou 119). 

In [58]:
# Connexion à la base de données MongoDB
client = MongoClient('mongodb://localhost', 27017)
db = client['climat_france'] 
aire_quality = db['aire_quality']
pollution_113 = db['pollution_113']
pollution_114 = db['pollution_114']
emission_region = db['emission_region']

In [59]:
#Le type de données à recupérer (112: indice qualité de l'air) 
id_data = 112 
# Date de début de la plage de recherche, la date de fin est au moment d'exécution de la requête
start_date = "01-01-2021"
# fichier contenant dans les codes insee, des communes, départements et régions.
# Mais aussi des colonnes supplémentaires pour chaque id_data pour vérifier les 
# communes/régions déja balayés pour l'id_data spécifique.
code_insee_file = "commune_insee.xlsx"
# Contients les informations des communes qui n'ont pas donnés de résultats pour un id_data donné.
# On les stock par précaution, mais n'ont présentement une utilité particulière.
trace_file = "empty_trace_com.csv"

In [ ]:
# Call the function to fetch air quality data for the Paris region since the start of last year
general_store(api_key, id_data, start_date, code_insee_file, trace_file)